In [19]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import holidays
from functions import business_days_between_dates

In [20]:
# Read recipe inputs
InputData_prep = dataiku.Dataset("InputData_prep")
df = InputData_prep.get_dataframe()

In [34]:
# Function to calculate business days between two dates, excluding weekends and holidays
def business_days_between_dates(start_date, end_date):
    # Ensure the dates are in datetime format
    start_date = pd.to_datetime(start_date, errors='coerce')
    end_date = pd.to_datetime(end_date, errors='coerce')
    
    # Check if either date is invalid, return 0 if so
    if pd.isna(start_date) or pd.isna(end_date):
        return 0
    
    # Generate a date range from start_date to end_date
    date_range = pd.date_range(start=start_date, end=end_date)
    
    # Filter out weekends (Saturday=5, Sunday=6)
    weekdays = date_range[~date_range.weekday.isin([5, 6])]
    
    # Get US holidays for the year of the start date
    us_holidays = holidays.US(years=start_date.year)
    
    # Convert the holidays keys (dates) to a pandas DatetimeIndex
    holiday_dates = pd.to_datetime(list(us_holidays.keys()))
    
    # Print the holidays for debugging purposes
    for date in holiday_dates:
        print(f"Holiday: {date}")
    
    # Remove the timezone from weekdays for comparison
    weekdays_naive = weekdays.tz_localize(None)
    
    # Filter out holidays (convert holidays to naive datetime as well)
    weekdays_without_holidays = weekdays_naive[~weekdays_naive.isin(holiday_dates)]
    
    # Return the number of weekdays excluding weekends and holidays
    return len(weekdays_without_holidays)

In [22]:
# Apply the function to calculate business days between 'Cash Start Date' and 'First Nearby Expiration'
def calculate_and_print_business_days(row):
    start_date = row['Cash Start Date']
    end_date = row['First Nearby Expiration']

    # Calculate the business days excluding weekends and holidays
    business_days = business_days_between_dates(start_date, end_date)

    # Print the result for this row
    print(f"Start Date: {start_date}, End Date: {end_date}, Business Days: {business_days}")

    return business_days

# Apply the function to each row and store the result in a new column
df['first_contract_days'] = df.apply(lambda row: calculate_and_print_business_days(row), axis=1)

Start Date: 2024-10-01 00:00:00+00:00, End Date: 2024-10-14 00:00:00+00:00, Business Days: 10


In [35]:
# Apply the function to calculate business days between 'Cash Start Date' and 'First Nearby Expiration'
def calculate_and_print_business_days(row):
    start_date = row["First Nearby Expiration"]
    end_date = row['Cash End Date']

    # Add one day to the "First Nearby Expiration" date
    start_date = pd.to_datetime(start_date) + pd.Timedelta(days=1)

    # Calculate the business days excluding weekends and holidays
    business_days = business_days_between_dates(start_date, end_date)

    # Print the result for this row
    print(f"Start Date: {start_date}, End Date: {end_date}, Business Days: {business_days}")

    return business_days

# Apply the function to each row and store the result in a new column
df['second_contract_days'] = df.apply(lambda row: calculate_and_print_business_days(row), axis=1)

Holiday: 2024-01-01 00:00:00
Holiday: 2024-05-27 00:00:00
Holiday: 2024-06-19 00:00:00
Holiday: 2024-07-04 00:00:00
Holiday: 2024-09-02 00:00:00
Holiday: 2024-11-11 00:00:00
Holiday: 2024-11-28 00:00:00
Holiday: 2024-12-25 00:00:00
Holiday: 2024-01-15 00:00:00
Holiday: 2024-02-19 00:00:00
Holiday: 2024-10-14 00:00:00
Start Date: 2024-10-15 00:00:00+00:00, End Date: 2024-11-30 00:00:00+00:00, Business Days: 32


In [0]:
# Write recipe outputs
DayCounts = dataiku.Dataset("DayCounts")
DayCounts.write_with_schema(df)